# Proposito
En este notebook estan los pasos a tomar para la etapa de **limpieza** del proceso de **Data Wrangling**. Este proceso recibe como entrada las conclusiones obtenidas en la etapa anterior: **estructuracion**.

Para este ejemplo, recibiremos como insumo un archivo CSV llamado `raw_data.csv`.

Finalmente, no olvidemos la necesidad del negocio. Necesitamos dar respuesta a la pregunta ¿Que grupo etario trae mas mal riesgo al banco?

# Objetivos
Al finalizar este notebook deberemos de haber:

✅ Verificar que los datos tienen un tipo de dato determinado.

✅ Eliminado los valores duplicados.

✅ Limpiado los valores extraños presentes.

In [1]:
import pandas as pd
from pandas import DataFrame

In [2]:
df = pd.read_csv(r"../data/raw_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0.1       4846 non-null   int64  
 1   Duration           4699 non-null   float64
 2   Checking account   2732 non-null   object 
 3   Credit amount      4576 non-null   object 
 4   Purpose            4686 non-null   object 
 5   Job                4635 non-null   float64
 6   Job                4635 non-null   float64
 7   Sex                4617 non-null   object 
 8   Sex                4617 non-null   object 
 9   Checking account   2732 non-null   object 
 10  Purpose            4686 non-null   object 
 11  csv                0 non-null      float64
 12  data_source        4844 non-null   object 
 13  Saving accounts    3788 non-null   object 
 14  Housing            4622 non-null   object 
 15  Risk               4715 non-null   object 
 16  Age                4705 

# Limpiando los valores nulos
En el proceso pasado: `estructuracion` hallamos que la columna: `csv` tenia todos sus datos como nulos, procedamos a eliminarla.

In [3]:
df: DataFrame = df.drop(columns=["csv"])

# Eliminando las columnas duplicadas
Asi mismo, en el proceso pasado hallamos que habian unas cuantas columnas duplicadas, eliminemoslas.

In [4]:
df: DataFrame = df.drop(columns=["Sex ", "Job ", "Purpose ", "Checking account "])

# Eliminando los valores extraños
Eliminemos todas las filas que contegan los valores extraños, identificados anteriormente. 

In [5]:
def drop_weird_values(df: DataFrame, weird_values: dict[str, list]):
    """
    Elimina los valores no esperados en cada columna según la lista de valores extraños.

    :param df: DataFrame de entrada.
    :param weird_values: Diccionario con los nombres de las columnas y sus valores no deseados.
    :return: DataFrame limpio.
    """
    df_cleaned: DataFrame = df.copy()  # Crear una copia para no modificar el original

    for column, values in weird_values.items():
        if column in df_cleaned.columns:
            df_cleaned = df_cleaned[
                ~df_cleaned[column].isin(values)
            ]  # Filtrar filas con valores no deseados

    return df_cleaned

In [6]:
weird_values: dict[str, list] = {
    "Credit amount": ["nan", "dfas", "qwretryet", "ttqweyuet"],
    "Age": ["hgd"],
    "Purpose": ["6", "3", "56", "356"],
    "Housing": ["563", "43", "356"],
    "Sex": ["353546"],
    "Risk": ["36", "5"],
}

df_cleaned: DataFrame = drop_weird_values(df, weird_values)

# Arreglando los tipos de datos
Finalmente, hagamos que las columnas tengan el tipo de dato que deberian de tener.

In [7]:
df_cleaned.columns

Index(['Unnamed: 0.1', 'Duration', 'Checking account', 'Credit amount',
       'Purpose', 'Job', 'Sex', 'data_source', 'Saving accounts', 'Housing',
       'Risk', 'Age', 'value', 'Unnamed: 0'],
      dtype='object')

In [8]:
def convert_to_expected_dtype(df: DataFrame, expected_dtypes: dict[str, type]):
    """
    Convierte las columnas del DataFrame a los tipos de datos especificados en expected_dtypes.

    :param df: DataFrame de entrada.
    :param expected_dtypes: Diccionario con los nombres de las columnas y los tipos esperados.
    :return: DataFrame con las columnas convertidas a los tipos correctos.
    """
    df_converted: DataFrame = df.copy()  # Crear una copia para no modificar el original

    for column, dtype in expected_dtypes.items():
        if column in df_converted.columns:
            try:
                df_converted[column] = df_converted[column].astype(dtype)
            except ValueError:
                print(
                    f"⚠️ No se pudo convertir la columna '{column}' a {dtype}. Posibles valores inválidos."
                )

    return df_converted

In [9]:
expected_dtypes: dict[str, type] = {
    "Unnamed: 0.1": "Int64",
    "Unnamed: 0": "Int64",
    "Duration": "Int64",
    "Checking account": "string",
    "Credit amount": "Int64",
    "Purpose": "string",
    "Job": "string",
    "Sex": "string",
    "data_source": "string",
    "Saving accounts": "string",
    "Housing": "string",
    "Risk": "string",
    "Age": "Int64",
    "value": "string",
}

df_converted: DataFrame = convert_to_expected_dtype(df_cleaned, expected_dtypes)

In [10]:
df_converted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4811 entries, 0 to 4845
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0.1      4811 non-null   Int64 
 1   Duration          4664 non-null   Int64 
 2   Checking account  2715 non-null   string
 3   Credit amount     4541 non-null   Int64 
 4   Purpose           4651 non-null   string
 5   Job               4600 non-null   string
 6   Sex               4582 non-null   string
 7   data_source       4809 non-null   string
 8   Saving accounts   3756 non-null   string
 9   Housing           4587 non-null   string
 10  Risk              4680 non-null   string
 11  Age               4670 non-null   Int64 
 12  value             4809 non-null   string
 13  Unnamed: 0        4650 non-null   Int64 
dtypes: Int64(5), string(9)
memory usage: 587.3 KB


Podemos ver que ahora los datos poseen un tipo de dato concreto y no `object` como lo solian hacer.

# Eliminemos las filas duplicadas

In [11]:
df_final: DataFrame = df_converted.drop_duplicates(keep="first")

# Conclusiones
Pudimos alcanzar cada uno de los objetivos propuestos al inicio de este notebook.
## Verificar que los datos tienen un tipo de dato determinado.
Los datos terminaron con una estructura definida y con un tipo de dato concreto.
## Eliminado de los valores duplicados.
Eliminamos todas las filas duplicadas.
## Limpiado de los valores extraños presentes.
Eliminamos todas las filas que tuvieran presentes valores no esperados para la columna.

In [12]:
df_final.to_csv(r"../data/cleaned_data.csv")  # guardemos los datos limpios para la siguiente fase